# 네거티브 샘플링을 이용한 Word2Vec 구현 

### (Skip-Gram with Negative Sampling, SGNS)

### 1. 네거티브 샘플링(Negative Sampling)
- Word2Vec 출력층에서는 소프트맥스 함수(확률 기반)를 지난 단어 집합 크기의 벡터와 실제값인 원-핫 벡터와의 오차를 구하고 , 임베딩 테이블에 있는 모든 단어에 대한 임베딩 벡터 값을 업데이트 해준다.
- 단어 집합의 크기가 클수록 작업이 무거워진다 (시간이 오래걸린다)

- Word2Vec은 역전파 과정에서 모든 단어의 임베딩 벡터값의 업데이트를 수행하지만, 중심단어와 주변단어가 "강아지","고양이", "귀여운" 과 같은 단어일 때 가중치를 업데이트 하는것은 비효율적인다.

- 네거티브 샘플링은, Word2Vec이 학습과정에서 전체 단어 집합이 아닌, 일부 단어 집합에만 집중할 수 있도록 하는 방법이다. 
    - 예) 주변단어가 '공양이', '귀여운' 일때 주변 단어가 아닌 단어를 랜덤하게 가져온다. 
        - 하나의 중심 단어에 대해서 전체 단어 집합보다 훨씬 작은 단어 집합을 만들고, 이진 분류 문제로 변환해준다.
        - 주변 단어들을 긍정, 랜덤 샘플링 된 단어들을 부정으로 분류
    - Word2Vec보다 연산량이 효율적임

### 2. SGNS (Skip-Gram with Negative Sampling)
- Skip-Gram은 중심단어로부터 주변 단어를 예측하는 모델이다.
    - 입력과 레이블을 각 인접하는 단어들로 구성을 했다면,
    - 인접하는 단어/주변단어인 경우 1, 단어 집합에서 랜덤으로 선택된 단어들은 레이블 0
<p align='center'><img src='https://wikidocs.net/images/page/69141/그림3.PNG'>
<img src='https://wikidocs.net/images/page/69141/그림4.PNG'>
<img src='https://wikidocs.net/images/page/69141/그림5.PNG'>
<img src='https://wikidocs.net/images/page/69141/그림6.PNG'>
<img src='https://wikidocs.net/images/page/69141/그림7.PNG'></p>

### 3. 20 뉴스그룹 데이터 전처리하기 

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# 하나의 샘플에 최소 단어 2개는 있어야 한다
    # 주변 단어, 중심 단어가 각각 1개씩은 있어야 하기 때문
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print("총 샘플 수  :", len(documents))

총 샘플 수  : 11314


In [3]:
# 토큰 제거 및 소문자화 & 정규화
news_df = pd.DataFrame({'document' : documents})
# 특수문자 제거 
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


/var/folders/jw/_38j243d5qq977z0w467cxq00000gn/T/ipykernel_89720/1801759761.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [4]:
# Null 이 있는지 확인하기
news_df.isnull().values.any()

False

In [5]:
# 빈 값 유무를 NaN 으로 바꾸어 확인
news_df.replace("", float("NaN"), inplace =True)
news_df.isnull().values.any()

True

In [6]:
news_df.dropna(inplace=True)
print("총 샘플 수 : ",len(news_df))

총 샘플 수 :  10995


In [7]:
# 불용어 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x
                                                if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

In [8]:
# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <=1 ]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print("총 샘플 수 : ", len(tokenized_doc))


총 샘플 수 :  10940


/Users/sunny/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [12]:
print( encoded[:2])

[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295], [1026, 532, 2, 60, 98, 582, 107, 800, 23, 79, 4522, 333, 7838, 864, 421, 3825, 458, 6488, 458, 2700, 4730, 333, 23, 9, 4731, 7262, 186, 310, 146, 170, 642, 1260, 107, 33568, 13, 985, 33569, 33570, 9471, 11491]]


In [13]:
# 단어 집합 크기 확인하기
vocab_size = len(word2idx)+1
print("단어 집합의 크기 : ", vocab_size)

단어 집합의 크기 :  64277


### 4. 네거티브 샘플링을 통한 데이터셋 구성
- 토큰화+정제+정규화+불용어 제거+정수 인코딩까지 전처리 완료
- 네거티브 샘플링을 통한 데이터셋을 구성할 차례
    - 시간이 많이 소요
    - 네거티브 샘플링을 위해 keras 의 skipgrams를 사용
    - 어떤 전처리가 수행되는지 결과를 확인을 상위 10개만 한다


In [14]:
from tensorflow.keras.preprocessing.sequence import skipgrams
# 네거티브 샘플링
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]

In [15]:
# 첫번쨰 샘플인 skip_grams[0], 
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}, {:s} ({:d})) -> {:d}".format(
        idx2word[pairs[i][0]], pairs[i][0],
        idx2word[pairs[i][1]], pairs[i][1],
        labels[i]))

(ignore (1979, subsidizing (15228)) -> 1
(lived (1148, teltone (36460)) -> 0
(ruin (9470, wilce (39445)) -> 0
(seem (207, story (603)) -> 1
(media (702, transportable (31530)) -> 0


- 주변 단어의 관계일 경우 -> 1
- 관계가 없는 경우       -> 0

In [16]:
print("전체 샘플 수 : ", len(skip_grams))

전체 샘플 수 :  10


encoded 중 상위 10개의 뉴스그룹 샘플에 대해서만 수행하였기 때문에 10이 출력된다.

- 그리고 10개의 뉴스그룹 샘플 각각은 수많은 중심 단어, 주변 단어의 쌍으로 된 샘플들을 갖고 있다.
- 첫번쨰 뉴스그룹 샘플이 가지고 있는 pairs 와 labels의 개수


In [17]:
# 첫번째 뉴스그룹 샘플에 대하여 생긴 Pairs와 labels의 개수
print(len(pairs))
print(len(labels))


2220
2220


In [18]:
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

### 5. Skip-Gram with Negative Sampling(SGNS)구현하기

In [25]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import dot,Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG


하이퍼파라미터인 임베딩 벡터의 차원은 100으로 고정
- 두 개의 임베딩 층을 추가

In [21]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)


Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-02-13 15:50:34.776374: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-13 15:50:34.776515: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


- 각 임베딩 테이블은 중심 단어와 주변 단어 각각을 위한 임베딩 테이블이다.
- 각 단어는 임베딩 테이블을 거쳐, 내적을 수행하고
    - 내적의 결과는 1/0 을 예측하기 위해 시그모이드 함수를 거친다

In [26]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1,1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs = [w_inputs, c_inputs], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       6427700     ['input_2[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 100)       6427700     ['input_3[0][0]']                
                                                                                              

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [30]:
for epoch in range(1, 6):
    loss = 0
    for _, elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [first_elem, second_elem]
        Y = labels
        loss += model.train_on_batch(X,Y)  
    print('Epoch :',epoch, 'Loss :',loss)

2022-02-13 15:55:14.646010: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-13 15:55:14.646092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-13 15:55:15.538803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-13 15:55:15.732432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch : 1 Loss : 4620.56647214666
Epoch : 2 Loss : 3649.586283698678
Epoch : 3 Loss : 3477.060306314379
Epoch : 4 Loss : 3274.9034411069006


### 6. 모델 결과 확인하기

In [ ]:
import gensim

f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embed_size))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

# 모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [ ]:
w2v.most_similar(positive=['soldiers'])

In [ ]:
w2v.most_similar(positive=['doctor'])

In [ ]:
w2v.most_similar(positive=['police'])

In [ ]:
w2v.most_similar(positive=['engine'])